# Load Libs

In [1]:
### Load Python Lib##### 
########
import numpy as np
from math import *
import matplotlib.pyplot as plt
from random import gauss, randint
from matplotlib import rc
from matplotlib.ticker import FixedLocator, MultipleLocator, FormatStrFormatter
import time
from scipy.special import spence
import scipy.optimize
from scipy import stats
import itertools
import iminuit

## ROOT
from rootpy.tree import Tree, TreeModel, FloatCol, IntCol
from rootpy.io import root_open
from root_numpy import root2array, tree2array, list_branches,  rec2array, fill_hist,array2hist
import ROOT
from ROOT import gSystem
gSystem.Load('libRooFit')

#import rootpy.plotting
from rootpy.extern.six.moves import range
from rootpy.plotting import Hist, Hist2D, Hist3D, HistStack, Legend, Canvas
from rootpy.interactive import wait
from rootpy.plotting import root2matplotlib as rplt
from rootpy.interactive import wait

#very import to enable this so the plots can be showed in the page
get_ipython().magic(u'matplotlib inline')

# Use LaTeX font.
plt.rc('text', usetex=True)
plt.rc('font',**{'family':'serif','serif':['Computer Modern Roman'],'size':20})

import matplotlib.font_manager as font_manager
font_prop = font_manager.FontProperties(size=12)

Deg2Rad = 3.1415926/180.0

/Users/yez/Library/Python/2.7/lib/python/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


ROOTError: level=3000, loc='TMacOSXSystem::DynamicPathName', msg='libRooFit[.so | .dll | .dylib | .sl | .dl | .a] does not exist in /Users/yez/.cache/rootpy/x86_64-53434/dicts:/Users/yez/.cache/rootpy/x86_64-53434/modules:/usr/local/lib:/usr/lib:/usr/local/root/lib:/usr/local/gfortran/lib:/Users/yez/work/analyzer-1.5.31:/Users/yez/work/LHAPDF6/lib:/Users/yez/work/github/XEMC/SRC:/usr/local/root/lib:/usr/lib::.:$ROOTSYS/lib:/usr/local/root/cint/cint/stl'

# Load MC Data from Root Tree

 ### Define ROOT Tree and Branches

In [3]:
## Branches in the tree if need to create new one

## Define all available variables here but only use ones needed
class Event(TreeModel):
    ## Kinematic Quantities
    Epsilon = FloatCol() 
    Qsq= FloatCol()
    t= FloatCol()
    W= FloatCol()
    x= FloatCol()
    y= FloatCol()
    z= FloatCol()
    phi_h= FloatCol()
    phi_S= FloatCol()
    EventWeight= FloatCol()
    weight= FloatCol()
    dilute= FloatCol()
#    PSF= FloatCol()
#    MM= FloatCol()
#    MM_res= FloatCol()
    N_Total= IntCol()
   
    ## XS and Asymmetry
    ZASigma_Lab= FloatCol()
    ZASig_T= FloatCol()
    ZASig_L= FloatCol()
    ZASig_LT= FloatCol()
    ZASig_TT= FloatCol()
    SSAsym= FloatCol()
    SineAsym= FloatCol()
    
    ## Electron Kine:
#    ele_ene= FloatCol()
#    ele_px= FloatCol()
#    ele_py= FloatCol()
#    ele_pz= FloatCol()
#    ele_mom= FloatCol()
#    ele_theta= FloatCol()
#    ele_phi= FloatCol()
    ## w/ detector resolution
#    ele_mom_res = FloatCol()
#    ele_theta_res = FloatCol()
#    ele_phi_res = FloatCol() 
    
    ## Pion Kine:
#    pim_ene= FloatCol()
#    pim_px= FloatCol()
#    pim_py= FloatCol()
#    pim_pz= FloatCol()
#    pim_mom= FloatCol()
#    pim_theta= FloatCol()
#    pim_phi= FloatCol()   
#    pim_mom_res= FloatCol()
#    pim_theta_res= FloatCol()
#    pim_phi_res= FloatCol()
    
    ## Recoil Proton Kine:
#    pro_ene= FloatCol()
#    pro_px= FloatCol()
#    pro_py= FloatCol()
#    pro_pz= FloatCol()
#    pro_mom= FloatCol()
#    pro_theta= FloatCol()
#    pro_phi= FloatCol()
    
    ## Neutron Target Kine:
#    tgt_px= FloatCol()
#    tgt_py= FloatCol()
#    tgt_pz= FloatCol()
#    vertexz = FloatCol()
    
    ## Acceptance:
    ele_acc_f = FloatCol()
    ele_acc_l = FloatCol()
    pim_acc_f = FloatCol()
    pim_acc_l = FloatCol()
    pro_acc_f = FloatCol()
    pro_acc_l = FloatCol()
    
#    = FloatCol()
#    = FloatCol()
#    = FloatCol()

### Load values from Tree

In [4]:
I = 3

In [5]:
## Define Rootfiles
filename="../rootfiles/dvmp_t%d.root"%I
file = root_open(filename,'r')
T = file.T

## 
branch_names = list_branches(filename, treename='T')
array  = tree2array(T, branches=branch_names)

x = array['x']
t = array['t']
W = array['W']
Qsq = array['Qsq']
Epsilon = array['Epsilon']

SSAsym = array['SSAsym']
SineAsym = array['SineAsym']
ZASig_L = array['ZASig_L']
ZASig_T = array['ZASig_T']
ZASig_LT = array['ZASig_LT']
ZASig_TT = array['ZASig_TT']
ZASigma_Lab = array['ZASigma_Lab']

dilute = array['dilute']
weight = array['weight']
EventWeight = array['EventWeight']
phi_H = array['phi_h']
phi_S = array['phi_S']

ele_acc_l = array['ele_acc_l']
ele_acc_f = array['ele_acc_f']
pim_acc_l = array['pim_acc_l']
pim_acc_f = array['pim_acc_f']
pro_acc_l = array['pro_acc_l']
pro_acc_f = array['pro_acc_f']

for i in range(0,len(weight)):
    weight[i] = EventWeight[i]*(ele_acc_l[i]+ele_acc_f[i])*pim_acc_f[i]*(pro_acc_l[i]+pro_acc_f[i])

#x = array['x']

# Asymmetry Fitting

## Define some constants

In [6]:
## Constants
time = 48 * 24 * 3600 * 1.0 #48days of 11 GeV beam time
target_factor = 0.865 #Effective neutron polarization
dilute_factor = 0.9 #should be one since it is exclude but give 0.9 for safty
target_pol = 0.6  #60% polarization
detector_eff = 0.85 #85% overall detector efficiency
Norm_Fact = time * pow(target_factor*dilute_factor, 2) * detector_eff
Nevnt = len(x)

TSA = np.zeros(Nevnt, dtype=float)
TSA_Bin = 0.0
Weight_Sum= 0.0
for i in range(0, Nevnt):
    #TSA[i] = -1.0 * Epsilon[i] * ZASig_L[i]/(Epsilon[i]*ZASig_L[i]+ZASig_T[i]) * SSAsym[i]
    TSA[i] = -1.0 * dilute[i] * SSAsym[i]

    #weight[i] *= Norm_Fact
    
    TSA_Bin += TSA[i] * weight[i]
    Weight_Sum += weight[i]
TSA_Bin /= Weight_Sum

N_out = Weight_Sum * Norm_Fact
dTSA_Bin = 1./target_pol * sqrt(1 - pow(target_pol*TSA_Bin, 2))/sqrt(N_out)

print '--- Average TSA for Bin#%d = %f +/-%f, from Evnt#=%d'%(I, TSA_Bin, dTSA_Bin, int(N_out))

--- Average TSA for Bin#3 = -0.220872 +/-0.003636, from Evnt#=206366


## ROOFIT

In [8]:
w = ROOT.RooWorkspace()

AttributeError: RooWorkspace

## Do the Maximum-Likelihood Fit with Function-1:

In [27]:
##################################################################
## define the asymmetry in terms of different angular modules
#model=lambda p,phiS, phiH:p[0]*(1+p[1]*np.sin(3*phiS + phiH) )
def TSA_Func(p, phi_S, phi_H):
    model = p[0] * (1. + p[1]*np.sin(3.*phi_S + phi_H) )
    #print '--Model = ', model
    
    return model
##################################################################


def like_model(parms):
    L = 0.0
    for phis in PHI_S:
        for phih in PHI_H:
            L+= np.log( TSA_Func(parms, phis, phih)/(2*np.pi*parms[0])  )
    #print L
    return -2.0*L

In [28]:
# step1: genereate events according to phi
nbins=60
grid=np.linspace(0,2*np.pi,nbins)
dphi=grid[1]-grid[0]
phimidS=0.5*(grid[:-1]+grid[1:])
phimidH = phimidS

p0=[1.0,TSA_Bin]
norm=2*np.pi*p0[0]  # integral of model from 0 to 2pi

W=[]
for phiS in phimidH:
    for phiH in phimidS:
        W.append( model(p0,phiS, phiH)*dphi/norm)
        
N=Nevnt # extect number of events
n=[int(w*N) for w in W]
PHI_S=[]
PHI_H=[]

for i in range(nbins-1):
    for j in range(nbins-1):
        phi1 = np.random.uniform(grid[i],grid[i+1],n[i*(nbins-1)+j])
        PHI_S=np.append(PHI_S, phi1)
        phi2 = np.random.uniform(grid[j],grid[j+1],n[i*(nbins-1)+j])
        PHI_H=np.append(PHI_H, phi2)
PHI_S=np.array(PHI_S, dtype=float)
PHI_H=np.array(PHI_H, dtype=float)

# step2: construct nll
#NLL=lambda p: -np.sum(np.log(model(p,PHI_S, PHI_H)/(2*np.pi*p[0])/(2*np.pi*p[0])))

# step3: make fit 
guess=[1,0.01]
# METH = 'L-BFGS-B'
METH = 'Nelder-Mead'
#METH = 'SLSQP'
#METH ='Powell' 
#METH ='CG'
#METH ='BFGS'
#METH ='Newton-CG'
#METH ='TNC'
#METH ='COBYLA'
res = scipy.optimize.minimize(like_model, guess, method=METH, jac=None, hess=None, tol=1e-6, callback=None, options={'disp':True})
pfit=res.x
print pfit


NameError: name 'model' is not defined

In [11]:
iter=0
results,cov,infodict,mesg,ier = scipy.optimize.leastsq(NLL, guess, full_output=1, ftol=1e-6)

A_UU = results[0]
A_UT = results[1]
simga = -1.0
chi2_AUT = (infodict['fvec'][0:]**2).sum()
ndof = len(infodict['fvec'])

if np.isinf(chi2_AUT) or np.isnan(chi2_AUT):
    chi2_AUT= -1.0
else:
    sigma = cov[1][1]

print "I = %d, MKL: A_avg = %f,  A_fit = %f, A_UU = %f, Sigma=%f, chi2 = %f/%d"%(I, TSA_Bin, A_UT, A_UU, sigma,chi2_AUT,ndof)


NameError: name 'NLL' is not defined